In [1]:
import tensorflow as tf
import random

/Users/seongjuhyeon/workspace/tensorflowZeroToAll/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
# To solve downloading files.
## https://github.com/tensorflow/tensorflow/issues/10779
## run the /Applications/Python 3.6/Install Certificates.command bash script to install newer certs.
mnist = input_data.read_data_sets('../MNIST_data/', one_hot=True)
print(mnist.train.num_examples)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
55000


In [6]:
# 0 ~ 9까지의 숫자 갯수(output숫자)
nb_classes = 10

In [9]:
# MNIST Data image of shape = 28 * 28 = 784
X = tf.placeholder(tf.float32, shape=[None, 28*28])
X_img = tf.reshape(X, [-1, 28, 28, 1]) # -1: N개, img 28x28x1 (black/white)
Y = tf.placeholder(tf.float32, shape=[None, nb_classes])

In [10]:
# dropout(keep_prob) rate 0.7~0.5 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

### L1 ImgIn shape=(?, 28, 28, 1)

In [14]:
# 3x3x1(color: black/white), 32 of filters
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
print(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1],
                   strides=[1,2,2,1], padding='SAME')
print(L1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
print(L1)

Tensor("Relu_2:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("dropout_1/mul:0", shape=(?, 14, 14, 32), dtype=float32)


### L2 ImgIn shape=(?, 14, 14, 32)

In [17]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
print(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1],
                   strides=[1,2,2,1], padding='SAME')
print(L2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)
print(L2)

Tensor("Relu_5:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_4:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("dropout_3/mul:0", shape=(?, 7, 7, 64), dtype=float32)


### L3 ImgIn shape=(?, 7, 7, 64)

In [19]:
W3 = tf.Variable(tf.random_normal([3,3,64,128], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding='SAME')
L3 = tf.nn.relu(L3)
print(L3)
L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1],
                   strides=[1,2,2,1], padding='SAME')
print(L3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
print(L3)
input_cnt = 4*4*128
L3_flat = tf.reshape(L3, [-1, input_cnt])

Tensor("Relu_7:0", shape=(?, 7, 7, 128), dtype=float32)
Tensor("MaxPool_6:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("dropout_5/mul:0", shape=(?, 4, 4, 128), dtype=float32)


### L4 FC(Fully Connected) 4x4x128 inputs -> 625 outputs

In [20]:
W4 = tf.get_variable("W4", shape=[input_cnt, 625],
                    initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

### L5 Final FC 625 inputs -> 10 outputs

In [21]:
# 10: 0~9 of numbers
W5 = tf.get_variable("W5", shape=[625, 10]
                    , initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L4, W5) + b5

### define cost/loss & optimizer

In [34]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
# Minimize cost
# learning_rate = 0.01로 하면 0.96~0.94 정도로 오히려 떨어짐...
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cost)

In [35]:
# Correct Prediction
prediction = tf.argmax(logits, 1)
real = tf.argmax(Y, 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, real), dtype=tf.float32))

In [36]:
# Prepare session
session = tf.Session()
session.run(tf.global_variables_initializer())

In [37]:
# Slice the data, epochs, batch_size
training_epochs = 15
batch_size = 100
total_iterator = int(mnist.train.num_examples / batch_size)

print(mnist.train.num_examples)
print(total_iterator)

55000
550


In [38]:
# Fit the train
for epoch in range(training_epochs):
    avg_cost = 0
    total_iterator = total_iterator
    
    for i in range(total_iterator):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.5}
        c_val, _ = session.run([cost, train], feed_dict=feed_dict)
        avg_cost = c_val / batch_size
    
    print("Epoch:", "%04d" % (epoch+1), ", Cost:", "{:.5f}".format(avg_cost))

print('Learning Finished!')
print("Accuracy:", session.run(accuracy,
                               feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))

Epoch: 0001 , Cost: 0.00292
Epoch: 0002 , Cost: 0.00119
Epoch: 0003 , Cost: 0.00130
Epoch: 0004 , Cost: 0.00123
Epoch: 0005 , Cost: 0.00288
Epoch: 0006 , Cost: 0.00049
Epoch: 0007 , Cost: 0.00063
Epoch: 0008 , Cost: 0.00016
Epoch: 0009 , Cost: 0.00051
Epoch: 0010 , Cost: 0.00042
Epoch: 0011 , Cost: 0.00008
Epoch: 0012 , Cost: 0.00043
Epoch: 0013 , Cost: 0.00100
Epoch: 0014 , Cost: 0.00079
Epoch: 0015 , Cost: 0.00004
Learning Finished!
Accuracy: 0.9934


In [39]:
# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", session.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", session.run(tf.argmax(logits, 1),
                                  feed_dict={X: mnist.test.images[r:r + 1], keep_prob: 1.0}))

Label:  [6]
Prediction:  [6]
